In [31]:
from autogen_agentchat.agents import AssistantAgent
import asyncio
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient
from autogen_core.models import UserMessage
from autogen_agentchat.ui import Console 
from autogen_core.memory import ListMemory,MemoryContent,MemoryMimeType 


In [ ]:
config_list = [
    {
    }
]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,  # Use 'cache_seed' instead of 'use_cache' (True becomes a seed value)
    "config_list": config_list,
}

In [34]:
user_memory = ListMemory()

In [8]:
await user_memory.add(MemoryContent(content="The weather should be in metric units", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="Meal recipe must be vegan", mime_type=MemoryMimeType.TEXT))

In [9]:
async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 °F and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 °C and Sunny."
    else:
        return f"Sorry, I don't know the weather in {city}."

In [30]:
import inspect
import importlib
import sys
from pprint import pprint

# Import the necessary library
from autogen_ext.models.azure import AzureAIChatCompletionClient

# Function to inspect the class and its methods
def inspect_azure_client():
    print("AzureAIChatCompletionClient Inspection:")
    print("-" * 50)
    
    # Get the class documentation
    print("Class Documentation:")
    print(AzureAIChatCompletionClient.__doc__)
    print("-" * 50)
    
    # Get the initialization parameters
    print("Initialization Parameters:")
    init_signature = inspect.signature(AzureAIChatCompletionClient.__init__)
    pprint(init_signature.parameters)
    print("-" * 50)
    
    # Try to access the _validate_config method to understand required fields
    print("Validation Method:")
    try:
        validate_method = getattr(AzureAIChatCompletionClient, "_validate_config")
        print(inspect.getsource(validate_method))
    except (AttributeError, TypeError):
        print("Could not access _validate_config method source code")
    print("-" * 50)
    
    # Check if there's a ModelInfo class or validation function
    print("Looking for ModelInfo class or validation function:")
    try:
        # Try to import the module that might contain ModelInfo
        azure_module = sys.modules.get('autogen_ext.models.azure')
        if azure_module:
            for name, obj in inspect.getmembers(azure_module):
                if name == "ModelInfo" or "model_info" in name.lower():
                    print(f"Found: {name}")
                    print(inspect.getsource(obj))
    except Exception as e:
        print(f"Error looking for ModelInfo: {e}")
    print("-" * 50)
    
    # Try to find the source file
    print("Source file location:")
    try:
        source_file = inspect.getsourcefile(AzureAIChatCompletionClient)
        print(f"Source file: {source_file}")
    except TypeError:
        print("Could not determine source file")
    print("-" * 50)

# Run the inspection
inspect_azure_client()

# Check the directory structure to find additional modules
print("Directory Structure:")
try:
    azure_path = importlib.import_module('autogen_ext.models.azure').__path__[0]
    print(f"Azure module path: {azure_path}")
    
    import os
    if os.path.exists(azure_path):
        print("Files in directory:")
        for file in os.listdir(azure_path):
            print(f"  - {file}")
except Exception as e:
    print(f"Error examining directory structure: {e}")

AzureAIChatCompletionClient Inspection:
--------------------------------------------------
Class Documentation:

    Chat completion client for models hosted on Azure AI Foundry or GitHub Models.
    See `here <https://learn.microsoft.com/en-us/azure/ai-studio/reference/reference-model-inference-chat-completions>`_ for more info.

    Args:
        endpoint (str): The endpoint to use. **Required.**
        credential (union, AzureKeyCredential, AsyncTokenCredential): The credentials to use. **Required**
        model_info (ModelInfo): The model family and capabilities of the model. **Required.**
        model (str): The name of the model. **Required if model is hosted on GitHub Models.**
        frequency_penalty: (optional,float)
        presence_penalty: (optional,float)
        temperature: (optional,float)
        top_p: (optional,float)
        max_tokens: (optional,int)
        response_format: (optional, literal["text", "json_object"])
        stop: (optional,List[str])
        

In [ ]:
assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=AzureAIChatCompletionClient(
        model="gpt-4o",
        endpoint="https://idkrag.openai.azure.com/",
        credential=AzureKeyCredential("7WYgbnwGu1U7OoVI2ZUdgAzp9zQ2uLCcpFR5xNCSBIYP9DcwLYRGJQQJ99BEACYeBjFXJ3w3AAABACOGKn3Y"),
        model_info={
            "json_output": False,
            "function_calling": True,  
            "vision": False,
            "family": "unknown",
        }
    ),
    tools=[get_weather],
    memory=[user_memory],
)

In [ ]:
# Run the agent with a task.
stream = assistant_agent.run_stream(task="What is the weather in New York?")
await Console(stream)


In [39]:
await assistant_agent._model_context.get_messages()


[UserMessage(content='What is the weather in New York?', source='user', type='UserMessage')]